This notebook demonstrates a basic implementation of the [Convex Parimutuel Call Auction Mechanism](http://web.stanford.edu/~yyye/cpcam-ec.pdf) (CPCAM). The implementation is formulated as a [Disciplined Convex Programming](http://dcp.stanford.edu/home) problem solved with the [Splitting Conic Solver](https://github.com/cvxgrp/scs). For a brief background on the CPCAM and related auction mechanisms, see [these slides](http://www.stat.uchicago.edu/~lekheng/meetings/mathofranking/slides/ye.pdf). Another introduction to CPCAM can be found [here](https://books.google.com/books?id=ass9QaLCU1MC&lpg=PA14&dq=convex%20parimutuel%20call%20market&pg=PA24#v=onepage&q=convex%20parimutuel%20call%20market&f=false).

For a longer discussion of such call auction or batch auction mechanisms, try my paper: [Smart Markets for Stablecoins](http://cdetr.io/smart-markets/). The [second example](http://cdetr.io/smart-markets/#LPAuctionChart) in that paper presents a batch auction solved by a Linear Programming (LP) method. As explained in the paper, the LP method does not guarantee the calculation of unique optimal clearing prices. The paper gives a set of example bids for which the LP method calculates non-unique state clearing prices of [0.40, 0.19, 0.22, 0.19].

The CPCAM method presented here does guarantee unique optimal clearing prices. To show this, CPCAM is applied to the same set of bids, and calculates (approximate) unique optimal clearing prices of [0.401961, 0.199836, 0.199836, 0.199836].



In [1]:
using Convex, SCS

In [2]:
s_total = 4; # number of market states / outcomes

# j_total - number of bids
# input data:
# a_i,j - trader j's order on state i
# pi_j - bid limit price
# q_j - bid quantity
#
# free variables, calculated by algo:
# x_j - bid order fill
# s_i - state prices

4

In [3]:
a = [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1;
    1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1;
    1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1;
    1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1;
    1 0 0 0]

pi = [0.25, 0.25, 0.25, 0.25,
      0.22, 0.22, 0.22, 0.22,
      0.19, 0.19, 0.19, 0.19,
      0.16, 0.16, 0.16, 0.16,
      0.40]


#a = [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 1;
#    1 0 0 0]

#pi = [0.10, 0.10, 0.10, 0.10,
#      0.40]

17-element Array{Float64,1}:
 0.25
 0.25
 0.25
 0.25
 0.22
 0.22
 0.22
 0.22
 0.19
 0.19
 0.19
 0.19
 0.16
 0.16
 0.16
 0.16
 0.4 

In [4]:
j_total = size(a,1)

17

In [5]:
s = Variable(s_total)
M = Variable(1)
x = Variable(j_total)
#q = Variable(j_total)
#pi = Variable(j_total)

Variable of
size: (17, 1)
sign: NoSign()
vexity: AffineVexity()

In [6]:
theta = [0.01, 0.01, 0.01, 0.01]
q = [20 for j in 1:j_total-1]
push!(q,100)

17-element Array{Int64,1}:
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
  20
 100

In [7]:
constraints = []

for i in 1 : s_total
    sum_over_j = sum([a[j,i] * x[j] for j in 1:j_total])
    constraints += sum_over_j + s[i] == M
end

constraints += [x >= 0; x <= q]
constraints += s >= 0

7-element Array{Constraint,1}:
 Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
 Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
 Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
 Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
 Constraint:
>= constraint
lhs: Variable of
size: (17, 1)
sign: NoSign()
vexity: AffineVexity()
rhs: 0
vexity: AffineVexity()                                        

In [8]:
problem = maximize(dot(pi, x) - M + sum([theta[i] * log(s[i]) for i in 1:s_total]), constraints)

Problem:
maximize AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: ConcaveVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
		Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
		Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
		Constraint:
== constraint
lhs: AbstractExpr with
head: +
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()

rhs: Variable of
size: (1, 1)
sign: NoSign()
vexity: AffineVexity()
vexity: AffineVexity()
		Constraint:
>= constraint
lhs: Variable of
size: (17, 1)
sign: NoSign()
vexity: Affi

In [9]:
solve!(problem, SCSSolver(verbose=3))

----------------------------------------------------------------------------
	SCS v1.1.5 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 94
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 27, constraints m = 55
Cones:	primal zero / dual free vars: 5
	linear vars: 38
	exp vars: 12, dual exp vars: 0
Setup time: 3.97e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  9.06e-04 
   100| 2.48e-03  2.84e-02  1.99e-01 -3.94e+00 -2.47e+00  0.00e+00  6.31e-03 
   200| 1.46e-03  1.23e-02  1.07e-01 -3.99e+00 -5.07e+00  0.00e+00  1.15e-02 
   300| 1.71e-03  4.82e-04  2.0

In [10]:
problem.optval

4.033912808329481

In [11]:
problem.status

:Optimal

In [12]:
#s.value

In [13]:
prices = theta ./ s.value

4x1 Array{Float64,2}:
 0.401961
 0.199836
 0.199836
 0.199836

In [14]:
x.value

17x1 Array{Float64,2}:
 -0.000286848
 20.0001     
 20.0001     
 20.0001     
 -0.000286705
 20.0001     
 20.0001     
 20.0001     
 -0.000286563
  9.13714e-5 
  9.13714e-5 
  9.13714e-5 
 -0.00028642 
  9.15138e-5 
  9.15138e-5 
  9.15138e-5 
 40.0292     